In [3]:
#import necessary packages
import pandas as pd
import numpy as np


In [327]:
#read csv
data=pd.read_csv("train.csv")
target='left'
# data1=data.values
# print(data.where(data["satisfaction_level"]==0.47).dropna())
# print(data.head())
keyValueMap={ j:i for i,j in enumerate(list(data.columns.values))}
features=[feature for feature in list(data.columns.values) if feature!=target]
cFeatures=['Work_accident','promotion_last_5years','sales','salary']
nFeatures=['satisfaction_level','last_evaluation','number_project','average_montly_hours','time_spend_company']
# print([(feature,np.unique(data[feature])) for feature in features])

In [99]:
def intrinsicCal(q):
    if q:
        return -q*np.log2(q)
    return 0

def entropy(q):
    if q>=1 or q<=0:
        return 0
    return -(q*np.log2(q)+(1-q)*np.log2(1-q))
def gini(q):
    return 2*q*(1-q)
def misClass(q):
    return min(q,1-q)
entropy(0.5)

1.0

In [313]:
#return the information gain of current Attribute with impurity type(entrop/gini/misclass) relative to target attribute
def infoGain(data,currentAttr,impType,targetAttr="left"):
#     print(data.shape[0],data.shape[1])
    totalRows=data.shape[0]
    
#     print("Target labels and target labels count")
#     print(np.unique(data[targetAttr],return_counts=True))
#     print()
    
    targetValues, targetValuesCount=np.unique(data[targetAttr],return_counts=True)
    currentValues, currentValuesCount=np.unique(data[currentAttr],return_counts=True)
#     print(targetValues,targetValuesCount,currentValues, currentValuesCount)
    
    #q is no employee will stay
#     print(",", targetValuesCount[0],totalRows)
    targetEntropy=impType(targetValuesCount[0]/(totalRows))
#     print("targetEntropy= ",targetEntropy)
    #(Si/S)*(entropy(target(=0)/Si)),(Si/S)*(entropy(target(=1)/Si)),.......
#     temp=[(currentValuesCount[j]/totalRows)*impType(np.unique(data.where(data[currentAttr]==i).dropna()[targetAttr],return_counts=True)[1][0]/currentValuesCount[j]) for j,i in enumerate(currentValues)]
    
#     temp=[((currentValuesCount[j]/totalRows)*impType(np.unique(data.where(data[currentAttr]==i).dropna()[targetAttr],return_counts=True)[1][0]/currentValuesCount[j])) for j,i in enumerate(currentValues)]
#     print(temp)
#     weightedEntropy=np.sum(temp)
#     print("weightedEntropy ",weightedEntropy)
    weightedEntropy=0
    intrinsic=0
    for j,i in enumerate(currentValues):
        weightedEntropy+= ((currentValuesCount[j]/totalRows)*impType(np.unique(data.where(data[currentAttr]==i).dropna()[targetAttr],return_counts=True)[1][0]/currentValuesCount[j]))
        intrinsic+=intrinsicCal(currentValuesCount[j]/totalRows)
#     print("weightedEntropy ",weightedEntropy)
    informationGain=targetEntropy-weightedEntropy
#     print("informationGain and intrinsic ",informationGain, intrinsic)
    return informationGain#/intrinsic
print(infoGain(data,'salary',entropy))
# print(infoGain(data,'promotion_last_5years',entropy))
# print(infoGain(data,'satisfaction_level',entropy))
# print(np.max([infoGain(data,feature,entropy) for feature in cFeatures]))

0.020887530533461263


In [306]:
def numericSplitInfoGain(data,currentAttr,numericValue, impType,targetAttr="left" ):
#     print(numericValue)
    totalRows=data.shape[0]
    currentAttrCol=data[currentAttr]
#     print(totalRows)
    targetValues, targetValuesCount=np.unique(data[targetAttr],return_counts=True)
    targetEntropy=impType(targetValuesCount[0]/(totalRows))
    numericValues=[] #all numberic values
    informationGains=[] #for all unique numeric values in data
    for numericValue in np.unique(data[currentAttr]):
#         print(numericValue)
        numericValues.append(numericValue)
        values1,lableCountsForLess=np.unique(data.where(currentAttrCol<=numericValue).dropna()[targetAttr],return_counts=True)
        values2,labelCountsForGreater=np.unique(data.where(currentAttrCol>numericValue).dropna()[targetAttr],return_counts=True)
        recordsLessThanNumeric=np.sum(lableCountsForLess)
        recordsGreaterThanNumeric=np.sum(labelCountsForGreater)
        print(numericValue,lableCountsForLess,labelCountsForGreater)
        a=0
        b=0
        intrinsic=0
        if recordsLessThanNumeric:
            a=(recordsLessThanNumeric/totalRows)*impType(lableCountsForLess[0]/recordsLessThanNumeric)
            intrinsic+=intrinsicCal(recordsLessThanNumeric/totalRows)
        if recordsGreaterThanNumeric:
            b=(recordsGreaterThanNumeric/totalRows)*impType(labelCountsForGreater[0]/recordsGreaterThanNumeric)
            intrinsic+=intrinsicCal(recordsGreaterThanNumeric/totalRows)
        weightedEntropy=a+b
        informationGain=targetEntropy-weightedEntropy
        informationGains.append(informationGain)
#     print(targetEntropy)
#     print(informationGain)
    return numericValues[np.argmax(informationGains)]
    
# data=data.sort_values(by="satisfaction_level", ascending=True, inplace=False)    
# s=np.unique(data['satisfaction_level'])
# s[np.argmax([numericSplitInfoGain(data,'satisfaction_level',value,entropy) for value in s ])]
numericSplitInfoGain(data,'satisfaction_level',value,entropy)
# numericSplitInfoGain(data,'satisfaction_level',0.89,entropy)

0.09 [147] [8563 2528]
0.1 [423] [8563 2252]
0.11 [674] [8563 2001]
0.12 [ 19 677] [8544 1998]
0.13 [ 56 680] [8507 1995]
0.14 [107 686] [8456 1989]
0.15 [158 687] [8405 1988]
0.16 [220 688] [8343 1987]
0.17 [269 692] [8294 1983]
0.18 [319 692] [8244 1983]
0.19 [362 695] [8201 1980]
0.2 [408 698] [8155 1977]
0.21 [462 699] [8101 1976]
0.22 [509 700] [8054 1975]
0.23 [547 701] [8016 1974]
0.24 [606 704] [7957 1971]
0.25 [625 705] [7938 1970]
0.26 [650 705] [7913 1970]
0.27 [664 709] [7899 1966]
0.28 [686 711] [7877 1964]
0.29 [717 711] [7846 1964]
0.3 [743 712] [7820 1963]
0.31 [776 726] [7787 1949]
0.32 [805 732] [7758 1943]
0.33 [827 735] [7736 1940]
0.34 [862 737] [7701 1938]
0.35 [887 739] [7676 1936]
0.36 [916 811] [7647 1864]
0.37 [951 963] [7612 1712]
0.38 [ 980 1067] [7583 1608]
0.39 [1004 1169] [7559 1506]
0.4 [1034 1298] [7529 1377]
0.41 [1066 1397] [7497 1278]
0.42 [1100 1480] [7463 1195]
0.43 [1133 1614] [7430 1061]
0.44 [1176 1722] [7387  953]
0.45 [1214 1850] [7349  825]
0

0.46

In [332]:
def numericSplitInfoGain(data,currentAttr,numericValue, impType,target="left"):
    totalRows=data.shape[0]
    data=data.sort_values(by=currentAttr, ascending=True, inplace=False)    
    data1=data.values;
    ones=np.sum(data[target])
    zeros=totalRows-ones
    targetEntropy=impType(ones/(totalRows))
    lableCountsForLess=[0,0]
    labelCountsForGreater=[0,0]
    key=0
    mx=-1
    i=0
    currentIndex=keyValueMap[currentAttr]
    targetIndex=keyValueMap[target]
    while i<totalRows:
        numericValue=data1[i][currentIndex]
        tl=data1[i][targetIndex]
        if tl==0:
            lableCountsForLess[0]+=1
            labelCountsForGreater[0]=zeros-lableCountsForLess[0]
            labelCountsForGreater[1]=ones-lableCountsForLess[1]
        else:
            lableCountsForLess[1]+=1
            labelCountsForGreater[0]=zeros-lableCountsForLess[0]
            labelCountsForGreater[1]=ones-lableCountsForLess[1]
            
        recordsLessThanNumeric=np.sum(lableCountsForLess)
        recordsGreaterThanNumeric=np.sum(labelCountsForGreater)
        a=0
        b=0
        intrinsic=0
        if recordsLessThanNumeric:
            a=(recordsLessThanNumeric/totalRows)*impType(lableCountsForLess[0]/recordsLessThanNumeric)
            intrinsic+=intrinsicCal(recordsLessThanNumeric/totalRows)
        if recordsGreaterThanNumeric:
            b=(recordsGreaterThanNumeric/totalRows)*impType(labelCountsForGreater[0]/recordsGreaterThanNumeric)
            intrinsic+=intrinsicCal(recordsGreaterThanNumeric/totalRows)
        weightedEntropy=a+b
        informationGain=targetEntropy-weightedEntropy
        if(i+1==totalRows or data1[i+1][currentIndex]!=numericValue):
            if(informationGain>=mx):
                mx=informationGain
                key=numericValue
        i+=1
    return key
numericSplitInfoGain(data,'average_montly_hours',value,entropy)

287

In [233]:
def bestInfoGainFeature(data,cFeatures,impType,targetAttr="left"):
    #info gains for categorical attributes    
    featureIndex=np.argmax([infoGain(data,feature,impType) for feature in cFeatures])
    bestFeature=cFeatures[featureIndex]
    return bestFeature
bestInfoGainFeature(data,cFeatures,entropy)

'salary'

In [225]:
def buildTree(filterdData,cFeatures,targetAttr="left"):
    #check if data is pure i.e all tuples have exact 1 type of label
    if(len(np.unique(filterdData[targetAttr]))<=1):
        print("hello= ",np.unique(filterdData[targetAttr])[0])
        print(filterdData)
        print(cFeatures)
        return np.unique(filterdData[targetAttr])[0]
    #check if all the categorical attributes had been used
    elif(len(cFeatures)==0):
        values,count=np.unique(filterdData[targetAttr],return_counts=True)
#         print(values[np.argmax(count)])
        return values[np.argmax(count)]
    #else grow the tree for all possible value of best feature
    #threby filtering the data on that value of best feature
    else:
        #find best feature
        #remove best feature from local list of features(cFeatures) for now
        #create a node with best feature and return it at the end
        #execute a for loop for all possible value of best feature (filtering on this value), and create a branch on
            #this value [best][value1] and receive the result of recursive call on this filtered data      
        bestF=bestInfoGainFeature(filterdData,cFeatures, entropy)
        node={bestF:{}}
        cFeatures=[feature for feature in cFeatures if feature!=bestF]
        for value in np.unique(filterdData[bestF]):
            #filter new datatable based on best feature [value]
#             print(bestF, value)
            filteredData=data.where(data[bestF]==value).dropna()
            node[bestF][value]=buildTree(filteredData,cFeatures)
        
    return node
    
tree=buildTree(data,cFeatures)



In [226]:
print(tree)

{'salary': {'high': {'sales': {'IT': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}, 1.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}}}, 'RandD': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}, 1.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}}}, 'accounting': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}, 1.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}}}, 'hr': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}, 1.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}}}, 'management': {'promotion_last_5years': {0.0: {'Work_accident': {0.0: 0.0, 1.0: 0.0}}, 1.0: {'Work_accident': {0.0: 0.0, 1.0: 0.0}}}}, 'marketing': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}, 1.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}}}, 'product_mng': {'Work_accident': {0.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}, 1.0: {'promotion_last_5years': {0.0: 0.0, 1.0: 0.0}}}